In [21]:
import torch
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation, AutoImageProcessor, BeitForSemanticSegmentation
backbone = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b4-finetuned-ade-512-512")
backbone.decode_head.classifier = torch.nn.Identity()
# import sys
image_processor = AutoImageProcessor.from_pretrained("nvidia/segformer-b4-finetuned-ade-512-512")
# backbone = BeitForSemanticSegmentation.from_pretrained("microsoft/beit-base-finetuned-ade-640-640")
# backbone.auxiliary_head.classifier = torch.nn.Identity()
# backbone.classifier = torch.nn.Identity()
backbone.decode_head.classifier = torch.nn.Identity()
# sys.path.append("./DIS/IS-Net")
# from models import *
# backbone = ISNetDIS().cuda()
# backbone.load_state_dict(torch.load("/home/wg25r/isnet-general-use.pth"))

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b4-finetuned-ade-512-512 and are newly initialized: ['segformer.encoder.test.bias', 'segformer.encoder.test.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
images=image_processor(images=torch.rand(1, 3, 224, 224), return_tensors="pt")
backbone(**images).logits.shape

torch.Size([1, 768, 128, 128])

In [23]:
# print(backbone(torch.randn(1, 3, 512, 512).cuda())[1][0].shape)

def model_fn(x):
    X = image_processor(images=x, return_tensors="pt").pixel_values.cuda()
    pred = backbone(pixel_values=X).logits
    return torch.nn.functional.interpolate(pred, size=(512, 512), mode="bilinear", align_corners=False)

In [24]:
backbone.eval()
import os

In [25]:
import torch
frames = os.listdir('/mnt/fastdata/dataset/baseline/office/input')[::10]
import tqdm
# features = torch.zeros((len(frames), 768, 320, 320))
len(frames)

205

In [26]:
from PIL import Image
from transformers import AutoImageProcessor
# image_processor = AutoImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
backbone = backbone.cuda()

In [29]:
import torchvision
import torchvision.transforms.v2
import numpy as np
first = None
import cv2
import tqdm
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter('difference.mp4', fourcc, 10, (512 * 3, 512))
# for i in tqdm.tqdm(range(len(difference))):
#     mask = (difference[i]) * 255 
#     mask = cv2.applyColorMap(mask.astype(np.uint8), cv2.COLORMAP_JET)
#     raw = cv2.imread(f'/mnt/fastdata/dataset/nbaseline/office/input/{frames[i]}')
#     raw = cv2.resize(raw, (640, 640))
#     frame = np.concatenate((raw, mask), axis=1)
#     video.write(frame)
# video.release() 

flatten_pred = []
flatten_gt = []

moving_avg = None

def all_close(a, b):
    return np.abs(a - b).max() < 10 

for i in tqdm.tqdm(range(len(frames))):
    with torch.no_grad():
        with torch.no_grad():
            frame = frames[i]
            img = Image.open(f'/mnt/fastdata/dataset/baseline/office/input/{frame}').resize((512, 512))
            gt = Image.open(f'/mnt/fastdata/dataset/baseline/office/groundtruth/{frame.replace("in","gt").replace("jpg","png")}').resize((512, 512)).convert("RGB")
            res = model_fn(img)
        if moving_avg is None:
            moving_avg = res
        else:
            moving_avg = 0.99 * moving_avg + 0.01 * res
        diff = 1 - torch.nn.functional.cosine_similarity(res, moving_avg, dim=1)
        mask = (diff.cpu().numpy()[0] > 0.2)* 255
        mask = cv2.applyColorMap(mask.astype(np.uint8), cv2.COLORMAP_JET)
        raw = cv2.imread(f'/mnt/fastdata/dataset/baseline/office/input/{frame}')
        raw = cv2.resize(raw, (512, 512))
        frame = np.concatenate((raw, mask, gt), axis=1)
        ROI = torch.tensor(~all_close(np.array(gt), 85))
        flatten_pred.append(diff.mean(0)[ROI].flatten().cpu())
        flatten_gt.append(torch.tensor(np.array(gt).mean(-1))[ROI].flatten())
        video.write(frame)
        
video.release()

  0%|          | 0/205 [00:00<?, ?it/s]/tmp/ipykernel_2781422/917118126.py:43: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  ROI = torch.tensor(~all_close(np.array(gt), 85))
100%|██████████| 205/205 [00:19<00:00, 10.75it/s]


In [ ]:
flatten_pred = torch.cat(flatten_pred)
flatten_gt = torch.cat(flatten_gt)
flatten_pred.shape, flatten_gt.shape

(torch.Size([38535168]), torch.Size([38535168]))

In [31]:
flatten_mask = flatten_pred > 0.2

In [ ]:
flatten_mask * flatten_gt


tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)

In [39]:
iou = (flatten_mask * flatten_gt).sum() / ((flatten_mask + flatten_gt).sum() - (flatten_mask * flatten_gt).sum())
iou

tensor(5.5053, dtype=torch.float64)

In [ ]:
# # mean = features[0]
# mean = torch.mean(features, dim=0)

# sim = torch.nn.CosineSimilarity(dim=1)(mean, features)


In [ ]:
difference = 1 - sim

In [ ]:
difference.shape

torch.Size([550, 128, 128])

In [ ]:
difference = difference.cpu().numpy()
# difference = difference.mean(1)


In [ ]:
# save difference into a mp4 video 
import cv2
import numpy as np
import os 
import tqdm

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter('difference.mp4', fourcc, 10, (320 * 2, 320))
for i in tqdm.tqdm(range(len(difference))):
    mask = (difference[i]) * 255 
    mask = cv2.applyColorMap(mask.astype(np.uint8), cv2.COLORMAP_JET)
    raw = cv2.imread(f'/mnt/fastdata/dataset/nbaseline/office/input/{frames[i]}')
    raw = cv2.resize(raw, (320, 320))
    frame = np.concatenate((raw, mask), axis=1)
    video.write(frame)
video.release() 

100%|██████████| 550/550 [00:01<00:00, 329.98it/s]


In [ ]:
# kmeans (kind of similar to mog) then chose the most common cluster as the background (even gpt knows this)